**Examen Fase II - Curso práctico Bioinformática**

*1. Recuerde cambiar el entorno de ejecución a un entorno de GPU (Runtime menu > Change Runtime Type > choose GPU)*

*2. Lea con calma el enunciado*

*3. Suerte*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install cmake --upgrade

In [ ]:
%%bash
wget ftp://ftp.gromacs.org/gromacs/gromacs-2021.tar.gz
tar xfz gromacs-2021.tar.gz
cd /content/gromacs-2021
mkdir /content/gromacs-2021/build
cd /content/gromacs-2021/build
cmake .. -DGMX_BUILD_OWN_FFTW=ON -DREGRESSIONTEST_DOWNLOAD=ON -DGMX_GPU=CUDA

In [ ]:
%%bash
cd /content/gromacs-2021/build
make -j 2
make install
source /usr/local/gromacs/bin/GMXRC

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
gmx

In [ ]:
!export GMXLIB=/usr/local/share/gromacs/top

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
gmx pdb2gmx -f protein.pdb -o protein.gro -p protein.top -i protein.itp -water spc -ignh -ff amber99sb-ildn
gmx editconf -f protein.gro -o protein_newbox.gro -c -d 1.0 -bt cubic
gmx solvate -cp protein_newbox.gro -o protein_solv.gro -p protein.top -cs spc216.gro
gmx grompp -f genion.mdp -c protein_solv.gro -p protein.top -o ions.tpr -maxwarn 2
echo "SOL" > options
echo " " >> options
gmx genion -s ions.tpr -o ions.gro -p protein.top -pname NA -nname CL -neutral < options

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
gmx grompp -f min.mdp -c ions.gro -p protein.top -o min.tpr
gmx mdrun -v -deffnm min 
gmx grompp -f nve.mdp -c min.gro -p protein.top -o nve.tpr -r min.gro
gmx mdrun -v -deffnm nve -nb gpu -pin on -ntmpi 2 -ntomp 1
gmx grompp -f nvt.mdp -c nve.gro -p protein.top -o nvt.tpr -r nve.gro
gmx mdrun -v -deffnm nvt -nb gpu -pin on -ntmpi 2 -ntomp 1
gmx grompp -f npt.mdp -c nvt.gro -p protein.top -o npt.tpr -r nvt.gro
gmx mdrun -v -deffnm npt -nb gpu -pin on -ntmpi 2 -ntomp 1

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
gmx editconf -f min.gro -o min.pdb -label A
gmx editconf -f nve.gro -o nve.pdb -label A
gmx editconf -f nvt.gro -o nvt.pdb -label A
gmx editconf -f npt.gro -o npt.pdb -label A

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
#Calculamos la energia potencial de los archivos de salida "min" 
echo "Potential" > options
echo " " >> options
gmx energy -f min.edr -o min_potential.xvg -xvg none < options

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
#Calculamos la temperatura de los archivos de salida "nve" 
echo "Temperature" > options
echo " " >> options
gmx energy -f nve.edr -o temperature_nve.xvg -xvg none < options

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
#Calculamos la temperatura de los archivos de salida "nve" 
echo "Temperature" > options
echo " " >> options
gmx energy -f nvt.edr -o temperature_nvt.xvg -xvg none < options

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /content/drive/MyDrive/Examen-FaseII/T1/
#Calculamos la temperatura de los archivos de salida "nve" 
echo "Density" > options
echo " " >> options
gmx energy -f npt.edr -o density.xvg -xvg none < options